In [ ]:
!pip install tf-keras-vis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 1.6 MB/s eta 0:00:00


In [ ]:
## load package
%reload_ext autoreload
%autoreload 2

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tf_keras_vis.utils import num_of_gpus
import matplotlib.pyplot as plt
%matplotlib inline
from keras.models import Sequential
from sklearn.model_selection import train_test_split
_, gpus = num_of_gpus()
print('Tensorflow recognized {} GPUs'.format(gpus))

Tensorflow recognized 1 GPUs


In [ ]:
from tensorflow.keras.applications import EfficientNetV2B0 as Model

model = Model(weights='imagenet', include_top=True)
model.compile(
        loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(lr=0.01), metrics=["accuracy"]
    )
model.summary()

29403144/29403144 [==============================] - 1s 0us/step


Model: "efficientnetv2-b0"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 rescaling (Rescaling)       (None, 224, 224, 3)          0         ['input_1[0][0]']             
                                                                                                  
 normalization (Normalizati  (None, 224, 224, 3)          0         ['rescaling[0][0]']           
 on)                                                                                              
                                                                                                  
 stem_conv (Conv2D)          (None, 112, 112, 32)         864       ['normalizatio

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
image_path = "/content/drive/MyDrive/Subset_1-40_81-120/Image_subset_032224"
SG_path = "/content/drive/MyDrive/Subset_1-40_81-120/Explanation/SG/"

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from keras.models import Sequential
from sklearn.model_selection import train_test_split


In [ ]:
from PIL import Image

def change_size_for_model(image):
    image = Image.fromarray(image)
    resized_image = image.resize((224, 224))
    resized_array = np.array(resized_image)
    return resized_array.astype(np.uint8)

def change_size_for_original(image, original_shape):
    image = Image.fromarray(image)
    resized_image = image.resize(original_shape) #resize image
    resized_array = np.array(resized_image)
    return resized_array.astype(np.uint8)

In [ ]:
start_idx =87
end_idx = 121

In [ ]:
list_of_dir =list()
for i in range(start_idx,end_idx):
  list_of_dir.append(str(i))

In [ ]:
from tf_keras_vis.utils.model_modifiers import ReplaceToLinear, ModelModifier

replace2linear = ReplaceToLinear()
def model_modifier_function(cloned_model):
    cloned_model.layers[-1].activation = tf.keras.activations.linear

In [ ]:
from tf_keras_vis.utils.scores import CategoricalScore


In [ ]:
from tensorflow.keras import backend as K
from tf_keras_vis.saliency import Saliency
# from tf_keras_vis.utils import normalize

# Create Saliency object.
saliency = Saliency(model,
                    model_modifier=model_modifier_function,
                    clone=True)

# Generate saliency map


In [ ]:
for j in range(len(list_of_dir)):
# for j in range(5):
  list_of_names = os.listdir(image_path+'/'+str(list_of_dir[j]))
  list_of_names.sort()
  original_image_array = list()

  for img_name in list_of_names:
    # Load the image
    image = Image.open(image_path+'/'+str(list_of_dir[j])+'/'+img_name)

    # Convert the image to a numpy array
    original_image_array.append(np.array(image))

  resized_image_array = list()
  for img in original_image_array:
    resized_image_array.append(change_size_for_model(img))

  resized_image_array = np.array(resized_image_array)

  ################
  # Your code
  try:
    result = model.predict(resized_image_array)
    saliency_map = list()
    for i in range(resized_image_array.shape[0]):
      X = resized_image_array[i]

      score = CategoricalScore([np.argmax(result[i])])

      saliency_map.append(saliency(score, X.astype(np.float32),smooth_samples=20,smooth_noise=0.10))
      # plt.imshow(saliency_map[i].reshape(224,224), cmap='gray')
      # plt.show()
      # plt.pause(1)

    ################

    folder_for_exp = SG_path + str(list_of_dir[j])
    if not os.path.exists(folder_for_exp):
        # The folder does not exist, so create it
        os.makedirs(folder_for_exp, exist_ok=True)
        print(f"Folder '{folder_for_exp}' was created.")
    else:
        # The folder already exists
        print(f"Folder '{folder_for_exp}' already exists.")

    for i in range(len(saliency_map)):
      a = 255*saliency_map[i].reshape(224,224)
      mod = np.percentile(a.flatten(),85)
      a[np.where(a >= mod)] = 255
      a[np.where(a < mod)] = 0

      # plt.figure()
      # X = resized_image_array[i]
      # plt.imshow(X, cmap='gray')
      # plt.imshow(a, cmap='binary_r', alpha=0.5)

      # plt.show()
      # plt.pause(1)
      plt.imsave(folder_for_exp+'/' +str(list_of_names[i]), a, cmap='binary_r')

  except:
     pass

1/1 [==============================] - 0s 27ms/step
Folder '/content/drive/MyDrive/Subset_1-40_81-120/Explanation/SG/88' was created.
1/1 [==============================] - 0s 27ms/step
Folder '/content/drive/MyDrive/Subset_1-40_81-120/Explanation/SG/89' was created.
1/1 [==============================] - 0s 51ms/step
Folder '/content/drive/MyDrive/Subset_1-40_81-120/Explanation/SG/90' was created.
1/1 [==============================] - 0s 46ms/step
Folder '/content/drive/MyDrive/Subset_1-40_81-120/Explanation/SG/91' was created.
1/1 [==============================] - 0s 28ms/step
Folder '/content/drive/MyDrive/Subset_1-40_81-120/Explanation/SG/92' was created.
1/1 [==============================] - 0s 28ms/step
Folder '/content/drive/MyDrive/Subset_1-40_81-120/Explanation/SG/93' was created.
1/1 [==============================] - 0s 30ms/step
Folder '/content/drive/MyDrive/Subset_1-40_81-120/Explanation/SG/94' was created.
1/1 [==============================] - 0s 28ms/step
Folder '/c